In [74]:
from qiskit_ibm_runtime import QiskitRuntimeService

token = '4c635c9b66b0ab2f37ce29d7bc33edc66f815847092ba236aa79748304b2cd4bbba6c35d8a4a6adc9b579b1990cfa8066f5220cbdc29044dc9bd59f1c1e1ea1b'
instance = 'ibm-q/open/main'

service = QiskitRuntimeService(
        channel='ibm_quantum',
        instance=instance,
        token=token
)

# Save your IBM Quantum API key in
# ~/.qiskit/qiskit-ibm.json
service.save_account(channel="ibm_quantum", 
                                  token=token,
                                  overwrite=True)

In [75]:
from qiskit import QuantumCircuit, ClassicalRegister

def create_quantum_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    # Apply Hadamard gate to each qubit
    for i in range(num_qubits):
        qc.h(i)
    # Apply CNOT gate between each pair of qubits
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    # Add classical registers and measurements
    cr = ClassicalRegister(num_qubits)
    qc.add_register(cr)
    qc.measure(range(num_qubits), range(num_qubits))
    return qc

# Specify the number of qubits
num_qubits = 6  # Change this value as needed
qc = create_quantum_circuit(num_qubits)

print(qc)

      ┌───┐          ┌─┐                              
 q_0: ┤ H ├──■───────┤M├──────────────────────────────
      ├───┤┌─┴─┐     └╥┘     ┌─┐                      
 q_1: ┤ H ├┤ X ├──■───╫──────┤M├──────────────────────
      ├───┤└───┘┌─┴─┐ ║      └╥┘     ┌─┐              
 q_2: ┤ H ├─────┤ X ├─╫───■───╫──────┤M├──────────────
      ├───┤     └───┘ ║ ┌─┴─┐ ║      └╥┘     ┌─┐      
 q_3: ┤ H ├───────────╫─┤ X ├─╫───■───╫──────┤M├──────
      ├───┤           ║ └───┘ ║ ┌─┴─┐ ║      └╥┘┌─┐   
 q_4: ┤ H ├───────────╫───────╫─┤ X ├─╫───■───╫─┤M├───
      ├───┤           ║       ║ └───┘ ║ ┌─┴─┐ ║ └╥┘┌─┐
 q_5: ┤ H ├───────────╫───────╫───────╫─┤ X ├─╫──╫─┤M├
      └───┘           ║       ║       ║ └───┘ ║  ║ └╥┘
c3: 6/════════════════╩═══════╩═══════╩═══════╩══╩══╩═
                      0       1       2       3  4  5 


In [76]:
print(service.instances())
print(service.backends())

['ibm-q/open/main']


[<IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_brisbane')>]


In [82]:
from qiskit_ibm_runtime import Session, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import state_fidelity
from qiskit_ibm_runtime import RuntimeJob


def run_program(backend_name, qc):
    # Get the backend
    backend = service.backend(backend_name)
    # set up pass_manager for backend
    pass_Manager = generate_preset_pass_manager(optimization_level=1, backend=backend)

    # Run pass_Manager thru transpiler (after 3/2024)
    qc_transpiled = pass_Manager.run(qc)

    # Run the Sampler job
    sampler = Sampler(mode=backend)
    job = sampler.run(pubs=[qc_transpiled])
    
    job_id = job.job_id()
    return job_id

backend_names = ['ibm_brisbane', 'ibm_sherbrooke', 'ibm_kyiv']

jobs_backend = []
for backend_name in backend_names:
    job_id = run_program(backend_name, qc)
    jobs_backend.append((job_id, backend_name))

print(jobs_backend)

In [86]:
from qiskit.quantum_info import state_fidelity

print(jobs_backend)

results = []
for job_id, backend_name in jobs_backend:
    print(f"Getting result for {backend_name}: {job_id}")
    job = service.job(job_id)
    result = job.result()
    results.append((result, backend_name))

# Compare results
for i in range(len(results)):
    for j in range(i + 1, len(results)):
        fidelity = state_fidelity(results[i][0].data.quasi_dists[0], results[j][0].data.quasi_dists[0])
        print(f"Fidelity between {results[i][1]} and {results[j][1]}: {fidelity}")

[('cxwq5gp4a290008xy8g0', 'ibm_brisbane'), ('cxwq5hywk6yg008j15e0', 'ibm_sherbrooke'), ('cxwq5ke4a290008xy8gg', 'ibm_kyiv')]
Getting result for ibm_brisbane: cxwq5gp4a290008xy8g0
Getting result for ibm_sherbrooke: cxwq5hywk6yg008j15e0
Getting result for ibm_kyiv: cxwq5ke4a290008xy8gg


AttributeError: 'PrimitiveResult' object has no attribute 'cr2'